In [2]:
import pandas as pd
import numpy as np
import tiktoken
from pypdf import PdfReader
import os
import pickle


import openai

#openai.api_key = 

In [9]:
########### NRGI ERFABLADE ##############
df = pd.DataFrame(columns=['Kilde','content', 'tokens'])

reader = PdfReader("Docs/Skadesvurdering_appen.pdf")

i = 0
for page in reader.pages:
    df.loc[i, 'Kilde'] = 'NRGI ERFAblade side ' + str(i+1)
    df.loc[i, 'content'] = page.extract_text()
    i+=1
    
df.head(5)

,Kilde,content,tokens
0,NRGI ERFAblade side 1,© NRGI BEST PRACTICE \nSKADESVURDERING ...,NaN
1,NRGI ERFAblade side 2,© NRGI 2 INDHOLDSFORTEGNELSE \n \nTAGKONSTR...,NaN
2,NRGI ERFAblade side 3,© NRGI 3 \nTAGKONSTRUKTION /-BELÆGNING/SKOR...,NaN
3,NRGI ERFAblade side 4,© NRGI 4 Asbestslam i tagrum Dato: 20-01-2...,NaN
4,NRGI ERFAblade side 5,© NRGI 5 Tagets restlevetid Dato: 10-02-20...,NaN


In [10]:
########### Håndbog ##############
reader = PdfReader("Docs/haandbog_for_beskikkede_bygningssagkyndige_final_rev_apr_2021.pdf")

i = df.shape[0]
j=0
for page in reader.pages:
    df.loc[i, 'Kilde'] = 'Håndbog for beskikkede bygningssagkyndige side ' + str(j+1)
    df.loc[i, 'content'] = page.extract_text()
    i+=1
    j+=1

In [11]:
########### Sikkerhedsstyrelsen guider ##############

directory = 'Docs/Sikkerhedsstyrelsen guider/'

for filename in os.listdir(directory):
    #print(os.path.join(directory, filename))
    reader = PdfReader(os.path.join(directory, filename))

    i = df.shape[0]
    for page in reader.pages:
        df.loc[i, 'Kilde'] = 'Sikkerhedsstyrelsen guider: ' + str(filename)
        df.loc[i, 'content'] = page.extract_text()
        i+=1

In [12]:
df.shape[0]

106

In [14]:
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

for i in range(df.shape[0]):
    df.loc[i, 'tokens'] = len(enc.encode(df.loc[i, 'content']))
    i+=1

#df = df.set_index(["Kilde"])
df.tail(5)

,Kilde,content,tokens
101,Sikkerhedsstyrelsen guider: guide_23_-_tagpapt...,\n \n \nGuide til vurdering af skader \n \n ...,1024
102,Sikkerhedsstyrelsen guider: guide_26_-_udvendi...,\n \n \nGuide til vurdering af skader \n \n ...,1468
103,Sikkerhedsstyrelsen guider: he-guide_33_-_vaad...,\n \n \nGuide til vurdering af skader \n \n ...,1114
104,Sikkerhedsstyrelsen guider: huseftersyn_guide_...,\n \n \nGuide til vurdering af skader \n \n ...,880
105,Sikkerhedsstyrelsen guider: huseftersyn_guide_...,\n \n \nGuide til vurdering af skader \n \n ...,1301


In [15]:
EMBEDDING_MODEL = "text-embedding-ada-002"
COMPLETIONS_MODEL = "gpt-3.5-turbo"

In [16]:
## This code was written by OpenAI: https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb

def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.content) for idx, r in df.iterrows()
    }

document_embeddings = compute_doc_embeddings(df)

# An example embedding:
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

0 : [-0.01169633585959673, -0.0019452517153695226, 0.020427802577614784, -0.03535144776105881, 0.015945034101605415]... (1536 entries)


In [17]:
# save dictionary to person_data.pkl file
with open('document_embeddings.pkl', 'wb') as fp:
    pickle.dump(document_embeddings, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [18]:
df.to_csv('df.csv', index=False, header=True)

In [9]:
## This code was written by OpenAI: https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb


def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_by_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [10]:
order_by_similarity("Can the commission implement acts for exchanging information?", document_embeddings)[:5]

[(0.7523785822075258, 65),
 (0.7487828850005422, 61),
 (0.7474731006913382, 56),
 (0.7474297860968954, 64),
 (0.7465414051992779, 59)]

In [11]:
order_by_similarity("Do I have permission to review my information?", document_embeddings)[:5]

[(0.7382950531856404, 83),
 (0.7376634867093262, 69),
 (0.73755125288604, 61),
 (0.7349670501204786, 54),
 (0.7331385095205205, 82)]

In [12]:
MAX_SECTION_LEN = 2000
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [13]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_by_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
        
    return chosen_sections, chosen_sections_len

In [14]:
def answer_with_gpt_4(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
) -> str:
    messages = [
        {"role" : "system", "content":"Du er en rådgiver chatbot der kun kan svare ud fra den kontekst du er blevet tilført her. Hvis du ikke kan svare på spørgsmålet skal du svare 'Svaret er ikke i ERFA bladene, håndbogen eller Sikkerhedsstyrelsens guider.'"}
    ]
    prompt, section_lenght = construct_prompt(
        query,
        document_embeddings,
        df
    )
    if show_prompt:
        print(prompt)

    context= ""
    for article in prompt:
        context = context + article 

    context = context + '\n\n --- \n\n + ' + query

    messages.append({"role" : "user", "content":context})
    response = openai.ChatCompletion.create(
        model=COMPLETIONS_MODEL,
        temperature=0.0,
        max_tokens=2000,
        messages=messages
        )

    return '\n' + response['choices'][0]['message']['content'], section_lenght

In [35]:
df.tail(50)

,Kilde,content,tokens
56,Håndbog for beskikkede bygningssagkyndige,Håndbog for beskikkede bygningssagkyndige \n1...,1043
57,Håndbog for beskikkede bygningssagkyndige,Håndbog for beskikkede bygningssagkyndige \n1...,796
58,Håndbog for beskikkede bygningssagkyndige,Håndbog for beskikkede bygningssagkyndige \n1...,984
59,Håndbog for beskikkede bygningssagkyndige,Håndbog for beskikkede bygningssagkyndige \n1...,312
60,Håndbog for beskikkede bygningssagkyndige,Håndbog for beskikkede bygningssagkyndige \n1...,1033
61,Håndbog for beskikkede bygningssagkyndige,Håndbog for beskikkede bygningssagkyndige \n1...,330
62,Håndbog for beskikkede bygningssagkyndige,Håndbog for beskikkede bygningssagkyndige \n1...,831
63,Håndbog for beskikkede bygningssagkyndige,Håndbog for beskikkede bygningssagkyndige \n1...,1117
64,Håndbog for beskikkede bygningssagkyndige,Håndbog for beskikkede bygningssagkyndige \n1...,1070
65,Håndbog for beskikkede bygningssagkyndige,Håndbog for beskikkede bygningssagkyndige \n1...,391


In [42]:
prompt = "Hvad koster nye vinduer?"
response, sections_tokens = answer_with_gpt_4(prompt, df, document_embeddings)
print(response)

Selected 3 document sections:
87
18
16

Svaret er ikke i ERFA bladene, håndbogen eller Sikkerhedsstyrelsens guider.
